# Classical Music Generation Model

## Dependencies

```bash
sudo apt-get install abcmidi timidity
pip install tensorflow==2.15
```
[Detailed tensorflow installation guide for gpu setup](https://www.tensorflow.org/install/pip).
You may need to install other packages listed in the imports if not installed already.

In [ ]:
import tensorflow as tf
import numpy as np
import os
from IPython import display as ipythondisplay
from tqdm import tqdm
from IPython.display import Audio
import subprocess
import tempfile
import regex as re



# Check for GPU. Remove this if you plan to run on CPU.
assert len(tf.config.list_physical_devices('GPU')) > 0

## Dataset



### Load dataset from file

In [ ]:
with open('training_data.txt', 'r') as file:
    songs = file.read()

### Find total unique characters in the dataset

In [ ]:
vocab = sorted(set(songs))
vocab_size = len(vocab)
print("There are", vocab_size, "unique characters in the dataset")

### Mapping the characters 

In [ ]:
# Mapping from character to unique index
char2idx = {u:i for i, u in enumerate(vocab)}

# Mapping from indices to characters
idx2char = np.array(vocab)

print(char2idx)
print(idx2char)

## Generate song

### Generate ABC notation text from the model

In [ ]:
model = tf.keras.models.load_model('saved_model/my_model')

model.summary()

def generate_text(model, start_string, generation_length=1000):

  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  text_generated = []

  model.reset_states()
  tqdm._instances.clear()

  for i in tqdm(range(generation_length)):
      predictions = model(input_eval)

      predictions = tf.squeeze(predictions, 0)

      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))


generated_text = generate_text(model, start_string="X", generation_length=1000)
print(generated_text)

### Function that converts ABC to MIDI to WAV

In [ ]:
def play_song(abcNotation, name):
    cwd = os.getcwd()

    # Convert ABC notation to MIDI
    midiFilePath = os.path.join(cwd, f'{name}.mid')
    with tempfile.NamedTemporaryFile(mode='w+', delete=False) as tmp_abc_file:
        tmp_abc_file.write(abcNotation)
        tmp_abc_file.flush()

        try:
            # Use abc2midi to convert ABC to MIDI
            command = ['abc2midi', tmp_abc_file.name, '-o', midiFilePath]
            subprocess.run(command, check=True)
            print(f"Conversion successful: {midiFilePath}")
        except subprocess.CalledProcessError as e:
            print(f"Error during conversion: {e}")
            return None

    # Convert MIDI to WAV using timidity
    wavFilePath = os.path.join(cwd, f'{name}.wav')
    try:
        # Use timidity to convert MIDI to WAV
        command = ['timidity', midiFilePath, '-Ow', '-o', wavFilePath]
        subprocess.run(command, check=True)
        print(f"WAV conversion successful: {wavFilePath}")
    except subprocess.CalledProcessError as e:
        print(f"Error during WAV conversion: {e}")
        return None

    # Play song
    return Audio(filename=wavFilePath)


### Extract and play the song(s)

In [ ]:
def extract_song_snippet(text):
    # Pattern to match each song starting with 'X:' until the next 'X:' or end of text
    pattern = r'(X:.*?)(?=\n\nX:|\Z)'
    search_results = re.findall(pattern, text, overlapped=True, flags=re.DOTALL)
    songs = [song for song in search_results]
    print("Found {} songs in text".format(len(songs)))
    return songs


generated_songs = extract_song_snippet(generated_text)

for i, song in enumerate(generated_songs):
  # Synthesize the waveform from a song
  output = play_song(song, f"generated_song_{i}")
  display(output)


The model isn't perfect, so you may need to rerun it if it doesn't work on the first try.
